<a href="https://colab.research.google.com/github/sthanikan2000/CS3121-Project/blob/main/DS_Proj_leaves_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
holi=pd.read_csv('holidays.csv')
df=pd.read_csv('leaves.csv')

In [ ]:
holi= pd.concat([pd.Series([holi.columns[0]]), holi[holi.columns[0]]], ignore_index=True)


In [ ]:
holi

0      2017-05-07
1      2017-05-14
2      2017-05-21
3      2017-05-28
4      2017-06-04
          ...    
116    2022-12-25
117    2022-12-29
118    2022-12-30
119    2022-07-13
120    2023-01-06
Length: 121, dtype: object

In [ ]:
df

,Employee_No,leave_date,Type,Applied Date,Remarks,apply_type
0,347,1/29/2020,Half Day,1/4/2022 15:33,\N,Anual
1,347,1/29/2020,Half Day,1/4/2022 15:33,\N,Anual
2,347,1/29/2020,Half Day,1/4/2022 15:36,\N,Anual
3,347,1/29/2020,Half Day,1/4/2022 15:36,\N,Anual
4,347,1/29/2020,Half Day,1/4/2022 18:02,\N,Anual
...,...,...,...,...,...,...
1013,596,1/27/2023,Full Day,1/31/2023 0:00,Personal,Casual
1014,596,1/28/2023,Half Day,1/31/2023 0:00,Personal,Casual
1015,2972,1/28/2023,Half Day,2/3/2023 14:00,Personal,Casual
1016,2972,1/30/2023,Full Day,2/3/2023 14:00,Personal,Casual


In [ ]:
df['leave_date'] = pd.to_datetime(df['leave_date'])
df['Applied Date'] = pd.to_datetime(df['Applied Date'])

mask= df["Applied Date"] > df["leave_date"]
df[mask]
# The above returned 974 records out of 1018
# Hence most of the leaves are not informed before taking a leave

,Employee_No,leave_date,Type,Applied Date,Remarks,apply_type
0,347,2020-01-29,Half Day,2022-01-04 15:33:00,\N,Anual
1,347,2020-01-29,Half Day,2022-01-04 15:33:00,\N,Anual
2,347,2020-01-29,Half Day,2022-01-04 15:36:00,\N,Anual
3,347,2020-01-29,Half Day,2022-01-04 15:36:00,\N,Anual
4,347,2020-01-29,Half Day,2022-01-04 18:02:00,\N,Anual
...,...,...,...,...,...,...
1013,596,2023-01-27,Full Day,2023-01-31 00:00:00,Personal,Casual
1014,596,2023-01-28,Half Day,2023-01-31 00:00:00,Personal,Casual
1015,2972,2023-01-28,Half Day,2023-02-03 14:00:00,Personal,Casual
1016,2972,2023-01-30,Full Day,2023-02-03 14:00:00,Personal,Casual


In [ ]:

df.drop(["Remarks","apply_type",'Applied Date'], axis=1, inplace=True)

df.drop_duplicates(inplace=True)


In [ ]:
mask=df['leave_date'].isin(holi)

In [ ]:
df=df[~mask] # Remove holidays from leaves
df

,Employee_No,leave_date,Type
0,347,2020-01-29,Half Day
5,1961,2022-01-24,Full Day
6,1961,2022-01-25,Full Day
7,2346,2022-01-10,Full Day
8,2346,2022-01-11,Full Day
...,...,...,...
1013,596,2023-01-27,Full Day
1014,596,2023-01-28,Half Day
1015,2972,2023-01-28,Half Day
1016,2972,2023-01-30,Full Day


In [ ]:
df.isnull().sum()

Employee_No    0
leave_date     0
Type           0
dtype: int64

In [ ]:
df.dtypes

Employee_No             int64
leave_date     datetime64[ns]
Type                   object
dtype: object

In [ ]:
df.nunique()

Employee_No     69
leave_date     361
Type             2
dtype: int64

In [ ]:
df

,Employee_No,leave_date,Type
0,347,2020-01-29,Half Day
5,1961,2022-01-24,Full Day
6,1961,2022-01-25,Full Day
7,2346,2022-01-10,Full Day
8,2346,2022-01-11,Full Day
...,...,...,...
1013,596,2023-01-27,Full Day
1014,596,2023-01-28,Half Day
1015,2972,2023-01-28,Half Day
1016,2972,2023-01-30,Full Day


In [ ]:
df3=df.copy()

df3.drop(["Type"], axis=1, inplace=True)

df3.drop_duplicates(inplace=True)


In [ ]:
df3
# returns 947 records as df. So, there are no situations like a employee take a half day and full day in same date.

,Employee_No,leave_date
0,347,2020-01-29
5,1961,2022-01-24
6,1961,2022-01-25
7,2346,2022-01-10
8,2346,2022-01-11
...,...,...
1013,596,2023-01-27
1014,596,2023-01-28
1015,2972,2023-01-28
1016,2972,2023-01-30


In [ ]:
grouped=df.groupby(['Employee_No', 'Type']).size().reset_index(name='Count')
grouped

,Employee_No,Type,Count
0,347,Full Day,16
1,347,Half Day,6
2,348,Full Day,5
3,348,Half Day,7
4,351,Full Day,4
...,...,...,...
115,2813,Full Day,1
116,2831,Half Day,1
117,2972,Full Day,1
118,2972,Half Day,2


In [ ]:
grouped.to_csv('leaves_preprocessed_200635E.csv', index=False)